# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [32]:
spaceship = pd.read_csv("spaceship.csv")
spaceship

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [33]:
spaceship = spaceship.dropna()

In [34]:
spaceship['Cabin'] = spaceship['Cabin'].astype(str)
spaceship['Cabin'] = spaceship['Cabin'].apply(lambda x: x[0] if len(x) > 0 else '')

C:\Users\usr\AppData\Local\Temp\ipykernel_19044\3012169917.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spaceship['Cabin'] = spaceship['Cabin'].astype(str)
C:\Users\usr\AppData\Local\Temp\ipykernel_19044\3012169917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spaceship['Cabin'] = spaceship['Cabin'].apply(lambda x: x[0] if len(x) > 0 else '')


In [35]:
spaceship = spaceship.drop(columns=['PassengerId', 'Name'])

In [36]:
non_numerical_columns = spaceship.select_dtypes(include=['object']).columns
spaceship = pd.get_dummies(spaceship, columns=non_numerical_columns, drop_first=True)

Now perform the same as before:
- Feature Scaling
- Feature Selection

**Perform Train Test Split**

In [37]:
X = spaceship.drop(columns=['Transported'])
y = spaceship['Transported']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
spaceship = spaceship.dropna()

In [39]:
normalizer = MinMaxScaler()
normalizer.fit(X_train)

MinMaxScaler()

In [40]:
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

In [41]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_train_norm

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
0,0.316456,0.000000,0.056116,0.000000,0.028650,0.030094,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.455696,0.000000,0.088015,0.135232,0.124911,0.000049,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.430380,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.468354,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.278481,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5279,0.392405,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5280,0.278481,0.003261,0.006507,0.000163,0.000000,0.018194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
5281,0.316456,0.000000,0.003857,0.000000,0.081355,0.198712,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5282,0.506329,0.000000,0.000704,0.000000,0.067074,0.068450,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [42]:
# Creamos el modelo Bagging (con reemplazo)
bagging_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(),  # Ahora usamos 'estimator' en lugar de 'base_estimator'
    n_estimators=100,     # Número de estimadores
    bootstrap=True,       # Bagging (con reemplazo)
    n_jobs=-1,            # Paralelización para velocidad
    random_state=42
)

# Entrenamos el modelo de Bagging
bagging_clf.fit(X_train, y_train)

# Realizamos predicciones con el conjunto de prueba
y_pred_bagging = bagging_clf.predict(X_test)

# Evaluamos el modelo
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
print(f"Accuracy Bagging: {accuracy_bagging:.4f}")

Accuracy Bagging: 0.8026


In [43]:
# Creamos el modelo Pasting (sin reemplazo)
pasting_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(),  # Ahora usamos 'estimator' en lugar de 'base_estimator'
    n_estimators=100,     # Número de estimadores
    bootstrap=False,      # Pasting (sin reemplazo)
    n_jobs=-1,            # Paralelización para velocidad
    random_state=42
)

# Entrenamos el modelo de Pasting
pasting_clf.fit(X_train, y_train)

# Realizamos predicciones con el conjunto de prueba
y_pred_pasting = pasting_clf.predict(X_test)

# Evaluamos el modelo
accuracy_pasting = accuracy_score(y_test, y_pred_pasting)
print(f"Accuracy Pasting: {accuracy_pasting:.4f}")


Accuracy Pasting: 0.7526


- Random Forests

In [44]:
# Creamos el modelo Random Forest
rf_clf = RandomForestClassifier(
    n_estimators=100,   # Número de árboles en el bosque
    random_state=42,    # Fijar semilla para reproducibilidad
    n_jobs=-1           # Paralelización para mayor velocidad
)

# Entrenamos el modelo Random Forest con los datos normalizados
rf_clf.fit(X_train_norm, y_train)

# Realizamos predicciones con el conjunto de prueba normalizado
y_pred_rf = rf_clf.predict(X_test_norm)

# Evaluamos el modelo
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Accuracy Random Forest: {accuracy_rf:.4f}")

Accuracy Random Forest: 0.8056


c:\Users\usr\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


- Gradient Boosting

In [45]:
# Creamos el modelo de Gradient Boosting
gb_clf = GradientBoostingClassifier(
    n_estimators=100,      # Número de árboles
    learning_rate=0.1,     # Tasa de aprendizaje
    random_state=42        # Fijar semilla para reproducibilidad
)

# Entrenamos el modelo Gradient Boosting con los datos normalizados
gb_clf.fit(X_train_norm, y_train)

# Realizamos predicciones con el conjunto de prueba normalizado
y_pred_gb = gb_clf.predict(X_test_norm)

# Evaluamos el modelo
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print(f"Accuracy Gradient Boosting: {accuracy_gb:.4f}")

Accuracy Gradient Boosting: 0.8071


c:\Users\usr\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


- Adaptive Boosting

In [46]:
# Creamos el modelo de AdaBoost
ada_clf = AdaBoostClassifier(
    n_estimators=100,     # Número de clasificadores débiles
    learning_rate=1.0,    # Tasa de aprendizaje
    random_state=42       # Fijar semilla para reproducibilidad
)

# Entrenamos el modelo AdaBoost con los datos normalizados
ada_clf.fit(X_train_norm, y_train)

# Realizamos predicciones con el conjunto de prueba normalizado
y_pred_ada = ada_clf.predict(X_test_norm)

# Evaluamos el modelo
accuracy_ada = accuracy_score(y_test, y_pred_ada)
print(f"Accuracy AdaBoost: {accuracy_ada:.4f}")

c:\Users\usr\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy AdaBoost: 0.7958


c:\Users\usr\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


Which model is the best and why?

- Gradient Boosting